# Internal Dependencies
<br>  

### References
- [Analyze java package metrics in a graph database](https://joht.github.io/johtizen/data/2023/04/21/java-package-metrics-analysis.html)
- [Calculate metrics](https://101.jqassistant.org/calculate-metrics/index.html)
- [Neo4j Python Driver](https://neo4j.com/docs/api/python-driver/current)

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plot
from neo4j import GraphDatabase

In [2]:
# Please set the environment variable "NEO4J_INITIAL_PASSWORD" in your shell 
# before starting jupyter notebook to provide the password for the user "neo4j". 
# It is not recommended to hardcode the password into jupyter notebook for security reasons.

driver = GraphDatabase.driver(uri="bolt://localhost:7687", auth=("neo4j", os.environ.get("NEO4J_INITIAL_PASSWORD")))
driver.verify_connectivity()

In [3]:
def get_cypher_query_from_file(cypherFileName):
    with open(cypherFileName) as file:
        return ' '.join(file.readlines())

In [4]:
def query_cypher_to_data_frame(filename : str, limit: int = 10_000):
    cypher_query_template = "{query}\nLIMIT {row_limit}"
    cypher_query = get_cypher_query_from_file(filename)
    cypher_query = cypher_query_template.format(query = cypher_query, row_limit = limit)
    records, summary, keys = driver.execute_query(cypher_query)
    return pd.DataFrame([r.values() for r in records], columns=keys)

In [5]:
def query_first_non_empty_cypher_to_data_frame(*filenames : str, limit: int = 10_000):
    """
    Executes the Cypher queries of the given files and returns the first result that is not empty.
    If all given file names result in empty results, the last (empty) result will be returned.
    By additionally specifying "limit=" the "LIMIT" keyword will appended to query so that only the first results get returned.
    """    
    result=pd.DataFrame()
    for filename in filenames:
        result=query_cypher_to_data_frame(filename, limit)
        if not result.empty:
            return result
    return result

In [6]:
#The following cell uses the build-in %html "magic" to override the CSS style for tables to a much smaller size.
#This is especially needed for PDF export of tables with multiple columns.

In [7]:
%%html
<style>
/* CSS style for smaller dataframe tables. */
.dataframe th {
    font-size: 8px;
}
.dataframe td {
    font-size: 8px;
}
</style>

## Artifacts

List the artifacts this notebook is based on. Different sorting variations help finding artifacts by their features and support larger code bases where the list of all artifacts gets too long.

Only the top 30 entries are shown. The whole table can be found in the following CSV report:  
`List_all_Java_artifacts`

In [8]:
artifacts = query_cypher_to_data_frame("../cypher/Internal_Dependencies/List_all_Java_artifacts.cypher")

### Table 1a - Top 30 artifacts with the highest package count

In [9]:
# Sort by number of packages descending
artifacts.sort_values(by=['packages','artifactName'], ascending=[False, True]).reset_index(drop=True).head(30)

artifactName  packages  types  incomingDependencies  \
0      axon-messaging-4.9.3.jar        64    786                     5   
1      axon-modelling-4.9.3.jar        10    156                     4   
2  axon-eventsourcing-4.9.3.jar         9    133                     3   
3           axon-test-4.9.3.jar         8     87                     0   
4  axon-configuration-4.9.3.jar         1     40                     0   
5      axon-disruptor-4.9.3.jar         1     22                     1   

   outgoingDependencies  
0                     0  
1                     1  
2                     2  
3                     3  
4                     4  
5                     3

### Table 1b - Top 30 artifacts with the highest type count

In [10]:
# Sort by number of types descending
artifacts.sort_values(by=['types','artifactName'], ascending=[False, True]).reset_index(drop=True).head(30)

artifactName  packages  types  incomingDependencies  \
0      axon-messaging-4.9.3.jar        64    786                     5   
1      axon-modelling-4.9.3.jar        10    156                     4   
2  axon-eventsourcing-4.9.3.jar         9    133                     3   
3           axon-test-4.9.3.jar         8     87                     0   
4  axon-configuration-4.9.3.jar         1     40                     0   
5      axon-disruptor-4.9.3.jar         1     22                     1   

   outgoingDependencies  
0                     0  
1                     1  
2                     2  
3                     3  
4                     4  
5                     3

### Table 1c - Top 30 artifacts with the highest number of incoming dependencies

The following table lists the top 30 artifacts that are used the most by other artifacts (highest count of incoming dependencies, highest in-degree).

In [11]:
# Sort by number of incoming dependencies descending
artifacts.sort_values(by=['incomingDependencies','artifactName'], ascending=[False, True]).reset_index(drop=True).head(30)

artifactName  packages  types  incomingDependencies  \
0      axon-messaging-4.9.3.jar        64    786                     5   
1      axon-modelling-4.9.3.jar        10    156                     4   
2  axon-eventsourcing-4.9.3.jar         9    133                     3   
3      axon-disruptor-4.9.3.jar         1     22                     1   
4  axon-configuration-4.9.3.jar         1     40                     0   
5           axon-test-4.9.3.jar         8     87                     0   

   outgoingDependencies  
0                     0  
1                     1  
2                     2  
3                     3  
4                     4  
5                     3

### Table 1d - Top 30 artifacts with the highest number of outgoing dependencies

The following table lists the top 30 artifacts that are depending on the highest number of other artifacts (highest count of outgoing dependencies, highest out-degree).

In [12]:
# Sort by number of outgoing dependencies descending
artifacts.sort_values(by=['outgoingDependencies','artifactName'], ascending=[False, True]).reset_index(drop=True).head(30)

artifactName  packages  types  incomingDependencies  \
0  axon-configuration-4.9.3.jar         1     40                     0   
1      axon-disruptor-4.9.3.jar         1     22                     1   
2           axon-test-4.9.3.jar         8     87                     0   
3  axon-eventsourcing-4.9.3.jar         9    133                     3   
4      axon-modelling-4.9.3.jar        10    156                     4   
5      axon-messaging-4.9.3.jar        64    786                     5   

   outgoingDependencies  
0                     4  
1                     3  
2                     3  
3                     2  
4                     1  
5                     0

### Table 1e - Top 30 artifacts with the lowest package count

In [13]:
# Sort by number of packages ascending
artifacts.sort_values(by=['packages','artifactName'], ascending=[True, True]).reset_index(drop=True).head(30)

artifactName  packages  types  incomingDependencies  \
0  axon-configuration-4.9.3.jar         1     40                     0   
1      axon-disruptor-4.9.3.jar         1     22                     1   
2           axon-test-4.9.3.jar         8     87                     0   
3  axon-eventsourcing-4.9.3.jar         9    133                     3   
4      axon-modelling-4.9.3.jar        10    156                     4   
5      axon-messaging-4.9.3.jar        64    786                     5   

   outgoingDependencies  
0                     4  
1                     3  
2                     3  
3                     2  
4                     1  
5                     0

### Table 1f - Top 30 artifacts with the lowest type count

In [14]:
# Sort by number of types ascending
artifacts.sort_values(by=['types','artifactName'], ascending=[True, True]).reset_index(drop=True).head(30)

artifactName  packages  types  incomingDependencies  \
0      axon-disruptor-4.9.3.jar         1     22                     1   
1  axon-configuration-4.9.3.jar         1     40                     0   
2           axon-test-4.9.3.jar         8     87                     0   
3  axon-eventsourcing-4.9.3.jar         9    133                     3   
4      axon-modelling-4.9.3.jar        10    156                     4   
5      axon-messaging-4.9.3.jar        64    786                     5   

   outgoingDependencies  
0                     3  
1                     4  
2                     3  
3                     2  
4                     1  
5                     0

### Table 1g - Top 30 artifacts with the lowest number of incoming dependencies

The following table lists the top 30 artifacts that are used the least by other artifacts (lowest count of incoming dependencies, lowest in-degree).

In [15]:
# Sort by number of incoming dependencies ascending
artifacts.sort_values(by=['incomingDependencies','artifactName'], ascending=[True, True]).reset_index(drop=True).head(30)

artifactName  packages  types  incomingDependencies  \
0  axon-configuration-4.9.3.jar         1     40                     0   
1           axon-test-4.9.3.jar         8     87                     0   
2      axon-disruptor-4.9.3.jar         1     22                     1   
3  axon-eventsourcing-4.9.3.jar         9    133                     3   
4      axon-modelling-4.9.3.jar        10    156                     4   
5      axon-messaging-4.9.3.jar        64    786                     5   

   outgoingDependencies  
0                     4  
1                     3  
2                     3  
3                     2  
4                     1  
5                     0

### Table 1h - Top 30 artifacts with the lowest number of outgoing dependencies

The following table lists the top 30 artifacts that are depending on the lowest number of other artifacts (lowest count of outgoing dependencies, lowest out-degree).

In [16]:
# Sort by number of outgoing dependencies ascending
artifacts.sort_values(by=['outgoingDependencies','artifactName'], ascending=[True, True]).reset_index(drop=True).head(30)

artifactName  packages  types  incomingDependencies  \
0      axon-messaging-4.9.3.jar        64    786                     5   
1      axon-modelling-4.9.3.jar        10    156                     4   
2  axon-eventsourcing-4.9.3.jar         9    133                     3   
3      axon-disruptor-4.9.3.jar         1     22                     1   
4           axon-test-4.9.3.jar         8     87                     0   
5  axon-configuration-4.9.3.jar         1     40                     0   

   outgoingDependencies  
0                     0  
1                     1  
2                     2  
3                     3  
4                     3  
5                     4

## Cyclic Dependencies

Cyclic dependencies occur when one package uses a class of another package and vice versa. 
These dependencies can lead to problems when one of these packages needs to be changed.

## Table 2a - Cyclic Dependencies Overview

Show the top 40 cyclic dependencies sorted by the most promising to resolve first. This is done by calculating the number of forward dependencies (first cycle participant to second cycle participant) in relation to backward dependencies (second cycle participant back to first cycle participant). The higher this rate (approaching 1), the easier it should be to resolve the cycle by focussing on the few backward dependencies.

Only the top 40 entries are shown. The whole table can be found in the following CSV report:  
`Cyclic_Dependencies`

**Columns:**
- *artifactName* identifies the artifact of the first participant of the cycle
- *packageName* identifies the package of the first participant of the cycle
- *dependentArtifactName* identifies the artifact of the second participant of the cycle
- *dependentPackageName* identifies the package of the second participant of the cycle
- *forwardToBackwardBalance* is between 0 and 1. High for many forward and few backward dependencies.
- *numberForward* contains the number of dependencies from the first participant of the cycle to the second one
- *numberBackward* contains the number of dependencies from the second participant of the cycle back to the first one
- *someForwardDependencies* lists some forward dependencies in the text format "type1 -> type2"
- *backwardDependencies* lists the backward dependencies in the format "type1 <- type2" that are recommended to get resolved

In [17]:
cyclic_dependencies = query_cypher_to_data_frame("../cypher/Cyclic_Dependencies/Cyclic_Dependencies.cypher")
cyclic_dependencies.head(40)

artifactName  \
0       axon-messaging-4.9.3   
1       axon-messaging-4.9.3   
2       axon-messaging-4.9.3   
3       axon-messaging-4.9.3   
4       axon-messaging-4.9.3   
5       axon-messaging-4.9.3   
6       axon-messaging-4.9.3   
7   axon-eventsourcing-4.9.3   
8       axon-messaging-4.9.3   
9       axon-messaging-4.9.3   
10      axon-messaging-4.9.3   
11      axon-messaging-4.9.3   
12      axon-messaging-4.9.3   
13      axon-messaging-4.9.3   
14  axon-eventsourcing-4.9.3   
15      axon-messaging-4.9.3   
16      axon-modelling-4.9.3   
17      axon-messaging-4.9.3   
18      axon-messaging-4.9.3   
19      axon-messaging-4.9.3   
20      axon-modelling-4.9.3   
21      axon-messaging-4.9.3   

                                          packageName  \
0                     org.axonframework.eventhandling   
1                     org.axonframework.queryhandling   
2                     org.axonframework.queryhandling   
3                     org.axonframework.eventhandling   
4                     org.axonframework.eventhandling   
5                          org.axonframework.deadline   
6                   org.axonframework.commandhandling   
7                     org.axonframework.eventsourcing   
8         org.axonframework.commandhandling.callbacks   
9                     org.axonframework.eventhandling   
10             org.axonframework.messaging.unitofwork   
11              org.axonframework.eventhandling.async   
12             org.axonframework.eventhandling.replay   
13    org.axonframework.serialization.upcasting.event   
14    org.axonframework.eventsourcing.eventstore.jdbc   
15                    org.axonframework.eventhandling   
16     org.axonframework.modelling.command.inspection   
17       org.axonframework.queryhandling.registration   
18                        org.axonframework.messaging   
19             org.axonframework.messaging.annotation   
20                   org.axonframework.modelling.saga   
21  org.axonframework.commandhandling.distributed....   

       dependentArtifactName  \
0       axon-messaging-4.9.3   
1       axon-messaging-4.9.3   
2       axon-messaging-4.9.3   
3       axon-messaging-4.9.3   
4       axon-messaging-4.9.3   
5       axon-messaging-4.9.3   
6       axon-messaging-4.9.3   
7   axon-eventsourcing-4.9.3   
8       axon-messaging-4.9.3   
9       axon-messaging-4.9.3   
10      axon-messaging-4.9.3   
11      axon-messaging-4.9.3   
12      axon-messaging-4.9.3   
13      axon-messaging-4.9.3   
14  axon-eventsourcing-4.9.3   
15      axon-messaging-4.9.3   
16      axon-modelling-4.9.3   
17      axon-messaging-4.9.3   
18      axon-messaging-4.9.3   
19      axon-messaging-4.9.3   
20      axon-modelling-4.9.3   
21      axon-messaging-4.9.3   

                                 dependentPackageName  \
0                           org.axonframework.tracing   
1           org.axonframework.messaging.responsetypes   
2                           org.axonframework.tracing   
3                         org.axonframework.messaging   
4              org.axonframework.messaging.annotation   
5                           org.axonframework.tracing   
6                           org.axonframework.tracing   
7          org.axonframework.eventsourcing.eventstore   
8                   org.axonframework.commandhandling   
9                     org.axonframework.serialization   
10                        org.axonframework.messaging   
11                    org.axonframework.eventhandling   
12                    org.axonframework.eventhandling   
13                    org.axonframework.eventhandling   
14  org.axonframework.eventsourcing.eventstore.jdb...   
15         org.axonframework.eventhandling.tokenstore   
16                org.axonframework.modelling.command   
17                    org.axonframework.queryhandling   
18                    org.axonframework.serialization   
19           org.axonframework.messaging.interceptors   
20         org.axonframework.mode

### Table 2b - Cyclic Dependencies Break Down

Lists packages with cyclic dependencies with every dependency in a separate row sorted by the most promising  dependency first.

Only the top 40 entries are shown. The whole table can be found in the following CSV report:  
`Cyclic_Dependencies_Breakdown`

**Columns in addition to Table 2a:**
- *dependency* shows the cycle dependency in the text format "type1 -> type2" (forward) or "type2<-type1" (backward)

In [18]:
cyclic_dependencies_breakdown = query_cypher_to_data_frame("../cypher/Cyclic_Dependencies/Cyclic_Dependencies_Breakdown.cypher",limit=40)
cyclic_dependencies_breakdown

artifactName                      packageName  \
0   axon-messaging-4.9.3  org.axonframework.eventhandling   
1   axon-messaging-4.9.3  org.axonframework.eventhandling   
2   axon-messaging-4.9.3  org.axonframework.eventhandling   
3   axon-messaging-4.9.3  org.axonframework.eventhandling   
4   axon-messaging-4.9.3  org.axonframework.eventhandling   
5   axon-messaging-4.9.3  org.axonframework.eventhandling   
6   axon-messaging-4.9.3  org.axonframework.eventhandling   
7   axon-messaging-4.9.3  org.axonframework.eventhandling   
8   axon-messaging-4.9.3  org.axonframework.eventhandling   
9   axon-messaging-4.9.3  org.axonframework.eventhandling   
10  axon-messaging-4.9.3  org.axonframework.eventhandling   
11  axon-messaging-4.9.3  org.axonframework.eventhandling   
12  axon-messaging-4.9.3  org.axonframework.eventhandling   
13  axon-messaging-4.9.3  org.axonframework.eventhandling   
14  axon-messaging-4.9.3  org.axonframework.eventhandling   
15  axon-messaging-4.9.3  org.axonframework.eventhandling   
16  axon-messaging-4.9.3  org.axonframework.eventhandling   
17  axon-messaging-4.9.3  org.axonframework.eventhandling   
18  axon-messaging-4.9.3  org.axonframework.eventhandling   
19  axon-messaging-4.9.3  org.axonframework.eventhandling   
20  axon-messaging-4.9.3  org.axonframework.queryhandling   
21  axon-messaging-4.9.3  org.axonframework.queryhandling   
22  axon-messaging-4.9.3  org.axonframework.queryhandling   
23  axon-messaging-4.9.3  org.axonframework.queryhandling   
24  axon-messaging-4.9.3  org.axonframework.queryhandling   
25  axon-messaging-4.9.3  org.axonframework.queryhandling   
26  axon-messaging-4.9.3  org.axonframework.queryhandling   
27  axon-messaging-4.9.3  org.axonframework.queryhandling   
28  axon-messaging-4.9.3  org.axonframework.queryhandling   
29  axon-messaging-4.9.3  org.axonframework.queryhandling   
30  axon-messaging-4.9.3  org.axonframework.queryhandling   
31  axon-messaging-4.9.3  org.axonframework.queryhandling   
32  axon-messaging-4.9.3  org.axonframework.queryhandling   
33  axon-messaging-4.9.3  org.axonframework.queryhandling   
34  axon-messaging-4.9.3  org.axonframework.queryhandling   
35  axon-messaging-4.9.3  org.axonframework.queryhandling   
36  axon-messaging-4.9.3  org.axonframework.queryhandling   
37  axon-messaging-4.9.3  org.axonframework.queryhandling   
38  axon-messaging-4.9.3  org.axonframework.queryhandling   
39  axon-messaging-4.9.3  org.axonframework.queryhandling   

   dependentArtifactName                       dependentPackageName  \
0   axon-messaging-4.9.3                  org.axonframework.tracing   
1   axon-messaging-4.9.3                  org.axonframework.tracing   
2   axon-messaging-4.9.3                  org.axonframework.tracing   
3   axon-messaging-4.9.3                  org.axonframework.tracing   
4   axon-messaging-4.9.3                  org.axonframework.tracing   
5   axon-messaging-4.9.3                  org.axonframework.tracing   
6   axon-messaging-4.9.3                  org.axonframework.tracing   
7   axon-messaging-4.9.3                  org.axonframework.tracing   
8   axon-messaging-4.9.3                  org.axonframework.tracing   
9   axon-messaging-4.9.3                  org.axonframework.tracing   
10  axon-messaging-4.9.3                  org.axonframework.tracing   
11  axon-messaging-4.9.3                  org.axonframework.tracing   
12  axon-messaging-4.9.3                  org.axonframework.tracing   
13  axon-messaging-4.9.3                  org.axonframework.tracing   
14  axon-messaging-4.9.3                  org.axonframework.tracing   
15  axon-messaging-4.9.3                  org.axonframework.tracing   
16  axon-messaging-4.9.3                  org.axonframework.tracing   
17  axon-messaging-4.9.3                  org.axonframework.tracing   
18  axon-messaging-4.9.3                  org.axonframework.tracing   
19  axon-messaging-4.9.3                  org.axonframework.tracing   
20  axon-messaging-

### Table 2c - Cyclic Dependencies Break Down - Backward Dependencies Only

Lists packages with cyclic dependencies with every dependency in a separate row sorted by the most promising  dependency first. This table only contains the backward dependencies from the second participant of the cycle back to the first one that are the most promising to resolve.

Only the top 40 entries are shown. The whole table can be found in the following CSV report:  
`Cyclic_Dependencies_Breakdown_BackwardOnly`

In [19]:
cyclic_dependencies_breakdown_backward = query_cypher_to_data_frame("../cypher/Cyclic_Dependencies/Cyclic_Dependencies_Breakdown_Backward_Only.cypher",limit=40)
cyclic_dependencies_breakdown_backward

artifactName                                      packageName  \
0       axon-messaging-4.9.3                  org.axonframework.eventhandling   
1       axon-messaging-4.9.3                  org.axonframework.queryhandling   
2       axon-messaging-4.9.3                  org.axonframework.queryhandling   
3       axon-messaging-4.9.3                  org.axonframework.eventhandling   
4       axon-messaging-4.9.3                  org.axonframework.eventhandling   
5       axon-messaging-4.9.3                  org.axonframework.eventhandling   
6       axon-messaging-4.9.3                  org.axonframework.eventhandling   
7       axon-messaging-4.9.3                  org.axonframework.eventhandling   
8       axon-messaging-4.9.3                       org.axonframework.deadline   
9       axon-messaging-4.9.3                org.axonframework.commandhandling   
10  axon-eventsourcing-4.9.3                  org.axonframework.eventsourcing   
11  axon-eventsourcing-4.9.3                  org.axonframework.eventsourcing   
12      axon-messaging-4.9.3      org.axonframework.commandhandling.callbacks   
13      axon-messaging-4.9.3      org.axonframework.commandhandling.callbacks   
14      axon-messaging-4.9.3                  org.axonframework.eventhandling   
15      axon-messaging-4.9.3                  org.axonframework.eventhandling   
16      axon-messaging-4.9.3                  org.axonframework.eventhandling   
17      axon-messaging-4.9.3           org.axonframework.messaging.unitofwork   
18      axon-messaging-4.9.3           org.axonframework.messaging.unitofwork   
19      axon-messaging-4.9.3           org.axonframework.messaging.unitofwork   
20      axon-messaging-4.9.3            org.axonframework.eventhandling.async   
21      axon-messaging-4.9.3            org.axonframework.eventhandling.async   
22      axon-messaging-4.9.3            org.axonframework.eventhandling.async   
23      axon-messaging-4.9.3           org.axonframework.eventhandling.replay   
24      axon-messaging-4.9.3           org.axonframework.eventhandling.replay   
25      axon-messaging-4.9.3           org.axonframework.eventhandling.replay   
26      axon-messaging-4.9.3  org.axonframework.serialization.upcasting.event   
27      axon-messaging-4.9.3  org.axonframework.serialization.upcasting.event   
28      axon-messaging-4.9.3  org.axonframework.serialization.upcasting.event   
29  axon-eventsourcing-4.9.3  org.axonframework.eventsourcing.eventstore.jdbc   
30  axon-eventsourcing-4.9.3  org.axonframework.eventsourcing.eventstore.jdbc   
31  axon-eventsourcing-4.9.3  org.axonframework.eventsourcing.eventstore.jdbc   
32  axon-eventsourcing-4.9.3  org.axonframework.eventsourcing.eventstore.jdbc   
33  axon-eventsourcing-4.9.3  org.axonframework.eventsourcing.eventstore.jdbc   
34  axon-eventsourcing-4.9.3  org.axonframework.eventsourcing.eventstore.jdbc   
35  axon-eventsourcing-4.9.3  org.axonframework.eventsourcing.eventstore.jdbc   
36  axon-eventsourcing-4.9.3  org.axonframework.eventsourcing.eventstore.jdbc   
37  axon-eventsourcing-4.9.3  org.axonframework.eventsourcing.eventstore.jdbc   
38  axon-eventsourcing-4.9.3  org.axonframework.eventsourcing.eventstore.jdbc   
39  axon-eventsourcing-4.9.3  org.axonframework.eventsourcing.eventstore.jdbc   

       dependentArtifactName  \
0       axon-messaging-4.9.3   
1       axon-messaging-4.9.3   
2       axon-messaging-4.9.3   
3       axon-messaging-4.9.3   
4       axon-messaging-4.9.3   
5       axon-messaging-4.9.3   
6       axon-messaging-4.9.3   
7       axon-messaging-4.9.3   
8       axon-messaging-4.9.3   
9       axon-messaging-4.9.3   
10  axon-eventsourcing-4.9.3   
11  axon-eventsourcing-4.9.3   
12      axon-messaging-4.9.3   
13      axon-messaging-4.9.3   
14      axon-messaging-4.9.3   
15      axon-messaging-4.9.3   
16      axon-messaging-4.9.3   
17      axon-messaging-4.9.3   
18      axon-messaging-4.9.3   
19      axon-messaging-4.9.3   
20      axon-messaging

## Interface Segregation Candidates

Well known from [Design Principles and Design Patterns by Robert C. Martin](http://staff.cs.utu.fi/~jounsmed/doos_06/material/DesignPrinciplesAndPatterns.pdf), the *Interface Segregation Principle* suggests that software components should have narrow, focused interfaces rather than large, general-purpose ones. The goal is to minimize the dependencies between components and increase modularity, flexibility, and maintainability.

Smaller, focused and purpose-driven interfaces

- make it easier to modify individual components without affecting the rest of the system.
- make it clearer which client is affected by which change.
- don’t force their clients to depend on methods they don’t need.
- reduce the scope of changes since a change to one component doesn’t affect others.
- lead to a more loosely coupled architecture that is easier to understand and maintain.

Reference: [Analyze java package metrics in a graph database](https://joht.github.io/johtizen/data/2023/04/21/java-package-metrics-analysis.html#interface-segregation)

### How to apply the results

If just one method of a type is used, especially in many places, then the result of this method can be used to call e.g. a method or constuct an object instead of using the whole object and then just calling that single method.

If there are a couple of methods that are used for a distinct purpose, those could be factored out into a separate interface. The original type can extended/implement the new interface so that there are no breaking changes. Then all the callers, that use only this group of methods, can be changed to the new interface.


### Table 4 - Top 40 most used combinations of methods

The following table shows the top 40 most used combinations of methods of larger types that might benefit from applying the *Interface Segregation Principle*. The whole table can be found in the CSV report `Candidates_for_Interface_Segregation`.

In [20]:
interface_segregation_candidates=query_cypher_to_data_frame("../cypher/Internal_Dependencies/Candidates_for_Interface_Segregation.cypher", limit=40)
interface_segregation_candidates

fullDependentTypeName  declaredMethods  \
0    org.axonframework.commandhandling.CommandMessage                9   
1   org.axonframework.eventhandling.DomainEventMes...               10   
2        org.axonframework.eventhandling.EventMessage                9   
3   org.axonframework.eventhandling.TrackedEventMe...               10   
4   org.axonframework.eventhandling.DomainEventMes...               10   
5        org.axonframework.eventhandling.EventMessage                9   
6           org.axonframework.messaging.ResultMessage                9   
7   org.axonframework.commandhandling.GenericComma...               14   
8   org.axonframework.deadline.GenericDeadlineMessage               11   
9         org.axonframework.eventhandling.ReplayToken               13   
10  org.axonframework.eventhandling.TrackedEventMe...               12   
11         org.axonframework.deadline.DeadlineMessage               10   
12  org.axonframework.eventhandling.DomainEventMes...               10   
13  org.axonframework.eventhandling.DomainEventMes...               10   
14  org.axonframework.eventhandling.GenericEventMe...               10   
15  org.axonframework.deadline.DefaultDeadlineMana...                8   
16  org.axonframework.common.transaction.NoTransac...                4   
17  org.axonframework.commandhandling.GenericComma...               15   
18  org.axonframework.modelling.command.inspection...               13   
19  org.axonframework.eventhandling.DomainEventMes...               11   
20  org.axonframework.eventhandling.GenericEventMe...               11   
21  org.axonframework.eventhandling.DomainEventMes...               10   
22  org.axonframework.eventhandling.GapAwareTracki...               10   
23       org.axonframework.eventhandling.EventMessage                9   
24       org.axonframework.messaging.MessageDecorator                9   
25   org.axonframework.eventhandling.TrackedEventData                5   
26  org.axonframework.eventhandling.tokenstore.Con...                5   
27      org.axonframework.eventhandling.TrackerStatus               17   
28  org.axonframework.modelling.command.inspection...               17   
29  org.axonframework.queryhandling.SimpleQueryUpd...               17   
30  org.axonframework.commandhandling.CommandMessa...               14   
31  org.axonframework.commandhandling.GenericComma...               14   
32         org.axonframework.deadline.DeadlineMessage               14   
33   org.axonframework.messaging.GenericResultMessage               14   
34  org.axonframework.messaging.annotation.Wrapped...               14   
35  org.axonframework.eventhandling.DomainEventMes...               13   
36        org.axonframework.eventhandling.ReplayToken               13   
37  org.axonframework.modelling.command.inspection...               13   
38  org.axonframework.queryhandling.SubscriptionQu...               12   
39  org.axonframework.commandhandling.GenericComma...               11   

                                    calledMethodNames  calledMethods  \
0                                    [getCommandName]              1   
1                                 [getSequenceNumber]              1   
2                       [getIdentifier, getTimestamp]              2   
3                                     [trackingToken]              1   
4   [getAggregateIdentifier, getType, getSequenceN...              3   
5                                     [getIdentifier]              1   
6                    [isExceptional, exceptionResult]              2   
7                            [asCommandResultMessage]              1   
8                                 [asDeadlineMessage]              1   
9                                 [createReplayToken]              1   
10                                    [trackingToken]              1   
11                                  [getDeadlineName]              1   
12                                          [getType]        

## Package Usage

### Table 5 - Types that are used by multiple packages

This table shows the top 40 packages that are used by the highest number of different packages. The whole table can be found in the CSV report `List_types_that_are_used_by_many_different_packages`.


In [21]:
types_used_by_many_packages=query_cypher_to_data_frame("../cypher/Internal_Dependencies/List_types_that_are_used_by_many_different_packages.cypher", limit=40)
types_used_by_many_packages

fullQualifiedDependentTypeName  \
0               org.axonframework.common.BuilderUtils   
1   org.axonframework.common.AxonConfigurationExce...   
2                 org.axonframework.messaging.Message   
3                org.axonframework.messaging.MetaData   
4        org.axonframework.eventhandling.EventMessage   
5   org.axonframework.messaging.unitofwork.UnitOfWork   
6          org.axonframework.serialization.Serializer   
7                     org.axonframework.common.Assert   
8   org.axonframework.common.transaction.Transacti...   
9    org.axonframework.serialization.SerializedObject   
10     org.axonframework.serialization.SerializedType   
11  org.axonframework.messaging.unitofwork.Current...   
12  org.axonframework.common.AxonNonTransientExcep...   
13  org.axonframework.eventhandling.DomainEventMes...   
14  org.axonframework.eventhandling.GenericEventMe...   
15  org.axonframework.messaging.annotation.Paramet...   
16      org.axonframework.eventhandling.TrackingToken   
17              org.axonframework.common.Registration   
18  org.axonframework.serialization.SimpleSerializ...   
19  org.axonframework.messaging.MessageHandlerInte...   
20               org.axonframework.common.ObjectUtils   
21              org.axonframework.tracing.SpanFactory   
22  org.axonframework.messaging.MessageDispatchInt...   
23   org.axonframework.commandhandling.CommandMessage   
24  org.axonframework.messaging.unitofwork.Default...   
25  org.axonframework.messaging.annotation.Handler...   
26           org.axonframework.common.ReflectionUtils   
27           org.axonframework.eventhandling.EventBus   
28              org.axonframework.lifecycle.Lifecycle   
29  org.axonframework.lifecycle.Lifecycle$Lifecycl...   
30  org.axonframework.common.transaction.NoTransac...   
31          org.axonframework.messaging.ResultMessage   
32             org.axonframework.common.DateTimeUtils   
33  org.axonframework.messaging.DefaultInterceptor...   
34       org.axonframework.messaging.InterceptorChain   
35          org.axonframework.tracing.NoOpSpanFactory   
36        org.axonframework.messaging.ScopeDescriptor   
37    org.axonframework.common.AxonTransientException   
38  org.axonframework.messaging.annotation.Classpa...   
39  org.axonframework.messaging.annotation.Classpa...   

                    dependentTypeName  \
0                        BuilderUtils   
1          AxonConfigurationException   
2                             Message   
3                            MetaData   
4                        EventMessage   
5                          UnitOfWork   
6                          Serializer   
7                              Assert   
8                  TransactionManager   
9                    SerializedObject   
10                     SerializedType   
11                  CurrentUnitOfWork   
12          AxonNonTransientException   
13                 DomainEventMessage   
14                GenericEventMessage   
15           ParameterResolverFactory   
16                      TrackingToken   
17                       Registration   
18             SimpleSerializedObject   
19          MessageHandlerInterceptor   
20                        ObjectUtils   
21                        SpanFactory   
22         MessageDispatchInterceptor   
23                     CommandMessage   
24                  DefaultUnitOfWork   
25                  HandlerDefinition   
26                    ReflectionUtils   
27                           EventBus   
28                          Lifecycle   
29        Lifecycle$LifecycleRegistry   
30               NoTransactionManager   
31                      ResultMessage   
32                      DateTimeUtils   
33            DefaultInterceptorChain   
34                   InterceptorChain   
35                    NoOpSpanFactory   
36                    ScopeDescriptor   
37             AxonTransientException   
38         ClasspathHandlerDefinition   
39  ClasspathParameterResolverFactory   

   

### Table 6 - Packages that are used by multiple artifacts

This table shows the top 30 artifacts that only use a few (compared to all existing) packages of another artifact.
The whole table can be found in the CSV report `ArtifactPackageUsage`.

In [22]:
used_packages_of_dependent_artifact=query_cypher_to_data_frame("../cypher/Internal_Dependencies/How_many_packages_compared_to_all_existing_are_used_by_dependent_artifacts.cypher",limit=30)
used_packages_of_dependent_artifact

artifactName     dependentArtifactName  dependentPackages  \
0      axon-modelling-4.9.3      axon-modelling-4.9.3                  6   
1           axon-test-4.9.3           axon-test-4.9.3                  5   
2      axon-messaging-4.9.3      axon-messaging-4.9.3                 41   
3  axon-eventsourcing-4.9.3  axon-eventsourcing-4.9.3                  7   

   dependentArtifactPackages  packageUsagePercentage  \
0                         10                0.600000   
1                          8                0.625000   
2                         64                0.640625   
3                          9                0.777778   

                  dependentFullQualifiedPackageNames  \
0  [org.axonframework.modelling.saga, org.axonfra...   
1  [org.axonframework.test, org.axonframework.tes...   
2  [org.axonframework.commandhandling, org.axonfr...   
3  [org.axonframework.eventsourcing.snapshotting,...   

                               dependentPackageNames  
0  [saga, jpa, repository, metamodel, command, in...  
1  [test, matchers, deadline, utils, eventscheduler]  
2  [commandhandling, messaging, callbacks, annota...  
3  [snapshotting, jpa, eventstore, statements, co...

### Table 7 - Types that are used by multiple artifacts

This table shows the top 30 types that only use a few (compared to all existing) types of another artifact. The whole table can be found in the CSV report `ClassesPerPackageUsageAcrossArtifacts`.

In [23]:
used_types_of_dependent_artifact=query_cypher_to_data_frame("../cypher/Internal_Dependencies/How_many_classes_compared_to_all_existing_in_the_same_package_are_used_by_dependent_packages_across_different_artifacts.cypher", limit=30)
used_types_of_dependent_artifact

artifactName dependentArtifactName  \
0       axon-modelling-4.9.3  axon-messaging-4.9.3   
1            axon-test-4.9.3  axon-messaging-4.9.3   
2   axon-eventsourcing-4.9.3  axon-messaging-4.9.3   
3   axon-eventsourcing-4.9.3  axon-modelling-4.9.3   
4   axon-eventsourcing-4.9.3  axon-modelling-4.9.3   
5   axon-eventsourcing-4.9.3  axon-messaging-4.9.3   
6            axon-test-4.9.3  axon-messaging-4.9.3   
7       axon-modelling-4.9.3  axon-messaging-4.9.3   
8       axon-modelling-4.9.3  axon-messaging-4.9.3   
9            axon-test-4.9.3  axon-messaging-4.9.3   
10  axon-eventsourcing-4.9.3  axon-messaging-4.9.3   
11  axon-eventsourcing-4.9.3  axon-messaging-4.9.3   
12           axon-test-4.9.3  axon-messaging-4.9.3   
13  axon-eventsourcing-4.9.3  axon-messaging-4.9.3   
14  axon-eventsourcing-4.9.3  axon-messaging-4.9.3   
15  axon-eventsourcing-4.9.3  axon-messaging-4.9.3   
16  axon-eventsourcing-4.9.3  axon-messaging-4.9.3   
17           axon-test-4.9.3  axon-modelling-4.9.3   
18           axon-test-4.9.3  axon-messaging-4.9.3   
19  axon-eventsourcing-4.9.3  axon-messaging-4.9.3   
20  axon-eventsourcing-4.9.3  axon-messaging-4.9.3   
21           axon-test-4.9.3  axon-messaging-4.9.3   
22           axon-test-4.9.3  axon-messaging-4.9.3   
23           axon-test-4.9.3  axon-messaging-4.9.3   
24      axon-modelling-4.9.3  axon-messaging-4.9.3   
25  axon-eventsourcing-4.9.3  axon-messaging-4.9.3   
26  axon-eventsourcing-4.9.3  axon-messaging-4.9.3   
27  axon-eventsourcing-4.9.3  axon-modelling-4.9.3   
28      axon-modelling-4.9.3  axon-messaging-4.9.3   
29  axon-eventsourcing-4.9.3  axon-messaging-4.9.3   

                                          packageName  \
0          org.axonframework.modelling.saga.metamodel   
1                     org.axonframework.test.matchers   
2        org.axonframework.eventsourcing.snapshotting   
3     org.axonframework.eventsourcing.eventstore.jdbc   
4   org.axonframework.eventsourcing.conflictresolu...   
5   org.axonframework.eventsourcing.conflictresolu...   
6               org.axonframework.test.eventscheduler   
7                 org.axonframework.modelling.command   
8       org.axonframework.modelling.command.legacyjpa   
9                              org.axonframework.test   
10  org.axonframework.eventsourcing.conflictresolu...   
11         org.axonframework.eventsourcing.eventstore   
12                    org.axonframework.test.matchers   
13    org.axonframework.eventsourcing.eventstore.jdbc   
14  org.axonframework.eventsourcing.eventstore.leg...   
15     org.axonframework.eventsourcing.eventstore.jpa   
16  org.axonframework.eventsourcing.eventstore.jdb...   
17                       org.axonframework.test.utils   
18                    org.axonframework.test.matchers   
19  org.axonframework.eventsourcing.conflictresolu...   
20  org.axonframework.eventsourcing.conflictresolu...   
21                       org.axonframework.test.utils   
22                      org.axonframework.test.server   
23              org.axonframework.test.eventscheduler   
24      org.axonframework.modelling.command.legacyjpa   
25  org.axonframework.eventsourcing.eventstore.jdb...   
26       org.axonframework.eventsourcing.snapshotting   
27         org.axonframework.eventsourcing.eventstore   
28      org.axonframework.modelling.command.legacyjpa   
29  org.axonframework.eventsourcing.conflictresolu...   

                      dependentPackage.fqn  dependentTypes  \
0          org.axonframework.eventhandling               1   
1          org.axonframework.eventhandling               1   
2          org.axonframework.eventhandling               1   
3      org.axonframework.modelling.command               1   
4      org.axonframework.modelling.command               1   
5          org.axonframework.eventhandling               2   
6          org.axonframework.eventhandling               2   
7          org.axonframework.eventhandling               2   
8          org.

### Table 8 - Duplicate package names across artifacts

This table shows the top 30 duplicate package names across artifacts. They are ordered by the number of duplicates descending.

This might lead to confusion, makes importing more error prone and might even lead to duplicate classes where only one of them will be loaded by the class loader. If a package is named the same way in two or more artifacts this even allows another artifact to access package protected classes, methods or members which might not be intended. 

The whole table can be found in the CSV report `DuplicatePackageNamesAcrossArtifacts`.

In [24]:
duplicate_package_names_across_artifacts=query_cypher_to_data_frame("../cypher/Artifact_Dependencies/Artifacts_with_duplicate_packages.cypher", limit=30)
duplicate_package_names_across_artifacts

Empty DataFrame
Columns: [packageName, duplicates, artifactNames]
Index: []

### Table 9 - Annotated elements

This table shows 30 most used Java Annotations including some examples where they are used.


In [25]:
annotated_elements=query_cypher_to_data_frame("../cypher/Java/Annotated_code_elements.cypher", limit=30)
annotated_elements

annotationName languageElement  \
0                            javax.annotation.Nonnull       Parameter   
1                                java.lang.Deprecated          Method   
2                           javax.annotation.Nullable       Parameter   
3                       java.lang.FunctionalInterface       Interface   
4                         java.lang.annotation.Target      Annotation   
5                      java.lang.annotation.Retention      Annotation   
6                             javax.persistence.Basic           Field   
7                            javax.annotation.Nonnull          Method   
8                           jakarta.persistence.Basic           Field   
9       com.fasterxml.jackson.annotation.JsonProperty       Parameter   
10                               java.lang.Deprecated           Class   
11                   java.beans.ConstructorProperties     Constructor   
12                    java.lang.annotation.Documented      Annotation   
13                               java.lang.Deprecated     Constructor   
14       com.fasterxml.jackson.annotation.JsonCreator     Constructor   
15                           javax.persistence.Column           Field   
16                         jakarta.persistence.Column           Field   
17                  org.axonframework.common.Priority           Class   
18                              javax.persistence.Lob           Field   
19                               javax.persistence.Id           Field   
20                              java.lang.SafeVarargs     Constructor   
21        com.fasterxml.jackson.annotation.JsonGetter          Method   
22                             jakarta.persistence.Id           Field   
23  org.axonframework.messaging.annotation.HasHand...      Annotation   
24                            jakarta.persistence.Lob           Field   
25                           javax.persistence.Entity           Class   
26                         jakarta.persistence.Entity           Class   
27  org.axonframework.messaging.annotation.Message...      Annotation   
28                 javax.persistence.MappedSuperclass           Class   
29                              java.lang.SafeVarargs          Method   

    numberOfAnnotatedElements  \
0                        1501   
1                          96   
2                          61   
3                          54   
4                          40   
5                          40   
6                          39   
7                          37   
8                          33   
9                          32   
10                         25   
11                         21   
12                         21   
13                         17   
14                         16   
15                         12   
16                         11   
17                         11   
18                         10   
19                         10   
20                          9   
21                          9   
22                          9   
23                          8   
24                          8   
25                          6   
26                          6   
27                          6   
28                          6   
29                          6   

                                             examples  
0   [org.axonframework.test.aggregate.ResultValida...  
1   [org.axonframework.test.aggregate.ResultValida...  
2   [org.axonframework.modelling.command.CreationP...  
3   [org.axonframework.test.matchers.FieldFilter, ...  
4   [org.axonframework.modelling.command.Aggregate...  
5   [org.axonframework.modelling.command.Aggregate...  
6   [org.axonframework.modelling.saga.repository.j...  
7   [org.axonframework.test.aggregate.AggregateTes...  
8   [org.axonframework.modelling.saga.repository.j...  
9   [org.axonframework.modelling.command.Aggregate...  
10  [org.axonframework.test.aggregate.StubAggregat...  
11  [org.axonframework.modelling.command.Aggregate...  
12  [org.axonframework.

### Table 10 - Distance distribution between dependent files

This table shows the file directory distance distribution between dependent files. Intuitively, the distance is given by the fewest number of change directory commands needed to navigate between a file and a dependency it uses. Those are aggregate to see how many dependent files are in the same directory, how many are just one change directory command apart, and so on.

In [26]:
query_first_non_empty_cypher_to_data_frame("../cypher/Internal_Dependencies/Get_file_distance_as_shortest_contains_path_for_dependencies.cypher",
                                           "../cypher/Internal_Dependencies/Set_file_distance_as_shortest_contains_path_for_dependencies.cypher", limit=20)

dependency.fileDistanceAsFewestChangeDirectoryCommands  \
0                                                  0        
1                                                  1        
2                                                  2        
3                                                  4        

   numberOfDependencies  numberOfDependencyUsers  numberOfDependencyProviders  \
0                  2331                      895                         1001   
1                    75                       70                           29   
2                  2265                      685                          371   
3                  1481                      292                          329   

                                            examples  
0  [/org/axonframework/test/aggregate uses /org/a...  
1  [/org/axonframework/test/deadline uses /org/ax...  
2  [/org/axonframework/test/deadline/DeadlineMana...  
3  [/org/axonframework/eventsourcing/EventSourcin...